## Librerias

In [1]:
from auxiliares import woe,iv

import cufflinks as cf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns 
import warnings

from scipy.stats import chi2_contingency

from sklearn.preprocessing import LabelEncoder
from varclushi import VarClusHi

sns.set()
cf.go_offline()
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

## Lectura de datos

In [2]:
train=pd.read_csv('~/train_discretizado.csv')
train.head()

id  loan_amnt  funded_amnt  funded_amnt_inv  int_rate  installment  \
0   9217698     6000.0       6000.0           6000.0     13.67       204.11   
1   5795876    27075.0      27075.0          27075.0     15.80       655.54   
2    480240    14000.0      14000.0          14000.0     13.11       472.43   
3  12175000    15000.0      15000.0          15000.0     15.31       359.30   
4   1457987     8700.0       8700.0           8700.0      8.90       276.26   

   annual_inc    dti  open_acc  revol_bal  revol_util  total_acc  tot_cur_bal  \
0     36000.0  15.20      12.0     7894.0        65.8       13.0      12871.0   
1     60780.0  18.34       9.0    21686.0        58.1       20.0     111385.0   
2     73000.0  21.34      16.0    17457.0        71.3       33.0      81398.5   
3     50000.0  32.48      26.0    16950.0        54.2       42.0      51457.0   
4     28500.0  14.03       7.0     3819.0        42.4       15.0      81398.5   

   total_rev_hi_lim  mths_between_issued_earliest_cr  \
0           12000.0                             51.0   
1           37300.0                            354.0   
2           22900.0                            114.0   
3           31300.0                            120.0   
4           22900.0                            221.0   

   collections_12_mths_ex_med  acc_now_delinq  tot_coll_amt  delinq_2yrs  \
0                         0.0             0.0           0.0          0.0   
1                         0.0             0.0        1115.0          0.0   
2                         0.0             0.0           0.0          0.0   
3                         0.0             0.0           0.0          2.0   
4                         0.0             0.0           0.0          0.0   

   inq_last_6mths  pub_rec    disc_loan_amnt_4  disc_funded_amnt_4  \
0             1.0      0.0     (500.0, 8000.0]     (500.0, 8000.0]   
1             0.0      0.0  (20000.0, 35000.0]  (20000.0, 35000.0]   
2             0.0      0.0  (12000.0, 20000.0]  (12000.0, 20000.0]   
3             1.0      0.0  (12000.0, 20000.0]  (12000.0, 20000.0]   
4             0.0      0.0   (8000.0, 12000.0]   (8000.0, 12000.0]   

  disc_funded_amnt_inv_4 disc_int_rate_4 disc_installment_4  \
0          (0.0, 8000.0]  (13.66, 16.49]    (15.67, 256.88]   
1     (19950.0, 35000.0]  (13.66, 16.49]  (565.62, 1409.99]   
2     (12000.0, 19950.0]  (10.99, 13.66]   (379.76, 565.62]   
3     (12000.0, 19950.0]  (13.66, 16.49]   (256.88, 379.76]   
4      (8000.0, 12000.0]   (5.42, 10.99]   (256.88, 379.76]   

    disc_annual_inc_4      disc_dti_4 disc_open_acc_4      disc_revol_bal_4  \
0   (1896.0, 45000.0]  (11.35, 16.87]    (10.0, 14.0]     (6413.0, 11769.0]   
1  (45000.0, 63000.0]  (16.87, 22.78]     (8.0, 10.0]  (20324.0, 2568995.0]   
2  (63000.0, 88956.5]  (16.87, 22.78]    (14.0, 84.0]    (11769.0, 20324.0]   
3  (45000.0, 63000.0]  (22.78, 39.99]    (14.0, 84.0]    (11769.0, 20324.0]   
4   (1896.0, 45000.0]  (11.35, 16.87]      (0.0, 8.0]         (0.0, 6413.0]   

  disc_revol_util_4 disc_total_acc_4   disc_tot_cur_bal_4  \
0      (57.6, 74.7]      (1.0, 17.0]       (0.0, 33597.0]   
1      (57.6, 74.7]     (17.0, 23.0]  (81398.5, 184569.0]   
2      (57.6, 74.7]    (32.0, 150.0]  (81398.5, 184569.0]   
3      (39.2, 57.6]    (32.0, 150.0]   (33597.0, 81398.5]   
4      (39.2, 57.6]      (1.0, 17.0]  (81398.5, 184569.0]   

  disc_total_rev_hi_lim_4 disc_mths_between_issued_earliest_cr_4  \
0          (0.0, 15000.0]                           (6.0, 133.0]   
1    (34300.0, 9999999.0]                         (238.0, 854.0]   
2      (22900.0, 34300.0]                           (6.0, 133.0]   
3      (22900.0, 34300.0]                           (6.0, 133.0]   
4      (22900.0, 34300.0]                         (176.0, 238.0]   

  disc_collections_12_mths_ex_med_4 disc_acc_now_delinq_4 disc_tot_coll_amt_4  \
0                       (0.0, 20.0]            (0.0, 5.0]    (0.0, 9152545.0]   
1                       (0.0, 20.0]     

## Selección de variables

In [3]:
vard=['term', 'grade', 'emp_length', 'home_ownership', 'verification_status', 'pymnt_plan',
       'purpose', 'addr_state', 'initial_list_status', ]

vars=[v for v in train.columns if v[:4]=='disc']+vard

um = 'id'

target='target'

### IV

In [4]:
aux=pd.Series({v:iv(train,v,target) for v in vars}).sort_values(ascending=False).reset_index()\
    .rename(columns={'index':'variable',0:'iv'})
best_iv=aux[aux['iv']>0.02]
best_iv.iplot(kind='bar',x='variable',y='iv',title='IV de las variables discretizadas',colors=['blue'])

### $\chi^2$ test

In [5]:
chi=pd.Series(dict(zip(best_iv['variable'],map(lambda x:chi2_contingency(pd.crosstab(train[x],train[target])).statistic,best_iv['variable']))))\
.reset_index().rename(columns={'index':'variable',0:'chi2'})
chi['cramer_V']=np.sqrt(chi['chi2']/train.shape[0])
chi=chi.sort_values('cramer_V',ascending=False)
chi.iplot(kind='bar',x='variable',y='cramer_V',title='Cramer\'s V de las variables discretizadas',colors=['blue'])

In [6]:
# variables con iv>.02 y cramer's V>.05
best=best_iv.merge(chi[chi['cramer_V']>.05],on='variable',how='inner').sort_values(by=['iv','cramer_V'],ascending=[False,False])
best

variable        iv         chi2  cramer_V
0                          disc_int_rate_10  0.321124  9414.127552  0.169830
1                           disc_int_rate_9  0.320822  9339.008238  0.169152
2                           disc_int_rate_7  0.313641  9240.430063  0.168256
3                           disc_int_rate_8  0.307107  9121.561675  0.167171
4                           disc_int_rate_6  0.302871  9004.756801  0.166097
5                           disc_int_rate_5  0.293599  8672.148962  0.163001
6                                     grade  0.286678  8803.688343  0.164232
7                           disc_int_rate_4  0.282296  8498.482963  0.161360
8                       disc_tot_cur_bal_10  0.056532  1742.689547  0.073069
9                  disc_total_rev_hi_lim_10  0.055305  1717.147881  0.072532
10                       disc_annual_inc_10  0.053488  1654.026346  0.071186
11                        disc_annual_inc_8  0.052933  1632.334536  0.070718
12                        disc_annual_inc_9  0.052612  1628.213406  0.070629
13                       disc_tot_cur_bal_8  0.052244  1589.834157  0.069791
14                        disc_annual_inc_7  0.052196  1611.332636  0.070262
15                        disc_annual_inc_6  0.051890  1609.596451  0.070224
16                  disc_total_rev_hi_lim_8  0.050924  1566.726664  0.069282
17                        disc_annual_inc_5  0.050707  1570.204265  0.069359
18                       disc_tot_cur_bal_5  0.050667  1558.341683  0.069097
19                        disc_annual_inc_4  0.050352  1560.369992  0.069142
20                       disc_tot_cur_bal_9  0.049003  1452.021943  0.066698
21                  disc_total_rev_hi_lim_5  0.048504  1503.307407  0.067866
22                       disc_tot_cur_bal_6  0.047938  1429.551628  0.066180
23                  disc_total_rev_hi_lim_9  0.047428  1420.783938  0.065977
24                  disc_total_rev_hi_lim_6  0.045366  1375.845883  0.064925
25                   disc_inq_last_6mths_10  0.044697  1490.329730  0.067572
26                    disc_inq_last_6mths_7  0.044697  1490.329730  0.067572
27                    disc_inq_last_6mths_9  0.044697  1490.329730  0.067572
28                    disc_inq_last_6mths_8  0.044697  1490.329730  0.067572
29                    disc_inq_last_6mths_6  0.044697  1490.329730  0.067572
30                    disc_inq_last_6mths_5  0.044697  1490.329730  0.067572
31                       disc_tot_cur_bal_7  0.043968  1276.546759  0.062538
32                  disc_total_rev_hi_lim_7  0.042619  1257.779303  0.062077
33                                     term  0.040271  1367.321681  0.064723
34                  disc_total_rev_hi_lim_4  0.037157  1103.647242  0.058149
35                       disc_tot_cur_bal_4  0.036260  1064.693495  0.057113
36                    disc_inq_last_6mths_4  0.035008  1108.614592  0.058280
37  disc_mths_between_issued_earliest_cr_10  0.027388   876.874794  0.051832
38   disc_mths_between_issued_earliest_cr_8  0.027386   871.551848  0.051674
39   disc_mths_between_issued_earliest_cr_9  0.026933   862.208601  0.051396
40                               disc_dti_9  0.026518   838.115049  0.050673
41                              disc_dti_10  0.026509   839.020162  0.050700
42   disc_mths_between_issued_earliest_cr_7  0.026478   841.052629  0.050762
43                               disc_dti_7  0.026143   827.159490  0.050341
44                               disc_dti_6  0.026132   825.611437  0.050294
45                               disc_dti_8  0.025920   820.416979  0.050135

### correlaciones

In [7]:
sns.heatmap(train[[f'woe_{var}' for var in best['variable']]].corr(),cmap='Blues')\
    .set_title('Correlacion entre las variables discretizadas')
plt.show()

### Cluster de variables

In [8]:
vc = VarClusHi(df=train,feat_list=[f'woe_{x}' for x in best['variable']]).varclus()

In [9]:
clus=vc.rsquare.sort_values(by=['Cluster','RS_Ratio']).reset_index(drop=True)
clus['Variable']=clus['Variable'].apply(lambda x: x[4:])

In [10]:
best=clus.merge(best,left_on='Variable',right_on='variable',how='inner').drop('variable',axis=1)\
    .sort_values(by=['Cluster','iv','cramer_V'],ascending=[True,False,False]).reset_index(drop=True)
best

Cluster                                 Variable    RS_Own     RS_NC  \
0         0                      disc_tot_cur_bal_10  0.919584  0.286567   
1         0                       disc_tot_cur_bal_8  0.938162  0.256510   
2         0                       disc_tot_cur_bal_5  0.873000  0.252862   
3         0                       disc_tot_cur_bal_9  0.950469  0.219109   
4         0                       disc_tot_cur_bal_6  0.933471  0.217389   
5         0                       disc_tot_cur_bal_7  0.901782  0.183501   
6         0                       disc_tot_cur_bal_4  0.824330  0.159913   
7         1                         disc_int_rate_10  0.970271  0.042741   
8         1                          disc_int_rate_9  0.970205  0.042120   
9         1                          disc_int_rate_7  0.967712  0.042659   
10        1                          disc_int_rate_8  0.976578  0.042986   
11        1                          disc_int_rate_6  0.967802  0.042453   
12        1                          disc_int_rate_5  0.952241  0.042014   
13        1                                    grade  0.921009  0.042096   
14        1                          disc_int_rate_4  0.926645  0.041081   
15        1                                     term  0.223307  0.016523   
16        2                   disc_inq_last_6mths_10  0.996240  0.041591   
17        2                    disc_inq_last_6mths_7  0.996240  0.041591   
18        2                    disc_inq_last_6mths_9  0.996240  0.041591   
19        2                    disc_inq_last_6mths_8  0.996240  0.041591   
20        2                    disc_inq_last_6mths_6  0.996240  0.041591   
21        2                    disc_inq_last_6mths_5  0.996240  0.041591   
22        2                    disc_inq_last_6mths_4  0.839325  0.036514   
23        3                       disc_annual_inc_10  0.989259  0.164878   
24        3                        disc_annual_inc_8  0.984990  0.164509   
25        3                        disc_annual_inc_9  0.984985  0.164707   
26        3                        disc_annual_inc_7  0.976274  0.163360   
27        3                        disc_annual_inc_6  0.979262  0.161220   
28        3                        disc_annual_inc_5  0.963803  0.158776   
29        3                        disc_annual_inc_4  0.955599  0.155844   
30        4                               disc_dti_9  0.986688  0.040675   
31        4                              disc_dti_10  0.991787  0.040786   
32        4                               disc_dti_7  0.981452  0.040465   
33        4                               disc_dti_6  0.983915  0.039794   
34        4                               disc_dti_8  0.987287  0.040553   
35        5  disc_mths_between_issued_earliest_cr_10  0.986143  0.078105   
36        5   disc_mths_between_issued_earliest_cr_8  0.985615  0.077530   
37        5   disc_mths_between_issued_earliest_cr_9  0.985193  0.077649   
38        5   disc_mths_between_issued_earliest_cr_7  0.981054  0.077593   
39        6                 disc_total_rev_hi_lim_10  0.921186  0.283514   
40        6                  disc_total_rev_hi_lim_8  0.946303  0.256724   
41        6                  disc_total_rev_hi_lim_5  0.855987  0.250961   
42        6                  disc_total_rev_hi_lim_9  0.948361  0.219830   
43        6                  disc_total_rev_hi_lim_6  0.930263  0.217771   
44        6                  disc_total_rev_hi_lim_7  0.902580  0.185784   
45        6                  disc_total_rev_hi_lim_4  0.825786  0.158421   

    RS_Ratio        iv         chi2  cramer_V  
0   0.112716  0.056532  1742.689547  0.073069  
1   0.083172  0.052244  1589.834157  0.069791  
2   0.169982  0.050667  1558.341683  0.069097  
3   0.063428  0.049003  1452.021943  0.066698  
4   0.085009  0.047938  1429.551628  0.066180  
5   0.120291  0.043968  1276.546759  0.062538  
6   0.209109  0.036260  1064.693495  0.057113  
7   0.031056  0.321124  9414.127552  0.169830  

In [11]:
best_var=best.groupby('Cluster').first()['Variable'].tolist()
best_var=[*map(lambda x:f'woe_{x}',best_var)]
best_var

['woe_disc_tot_cur_bal_10',
 'woe_disc_int_rate_10',
 'woe_disc_inq_last_6mths_10',
 'woe_disc_annual_inc_10',
 'woe_disc_dti_9',
 'woe_disc_mths_between_issued_earliest_cr_10',
 'woe_disc_total_rev_hi_lim_10']